In [3]:
# Method A: Empty dataset and add items.
import datasets # type: ignore

ds = datasets.Dataset.from_dict({})
n = 1_000_000

for i in range(n):
    ds = ds.add_item({'question': 'What is your name?', 'normal': str(i)})

print(ds)
print(ds[0])
print(ds[-1])

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fae226d2a40>>
Traceback (most recent call last):
  File "/home/zel/ml-projects/ShortGPT/.venv-shortgpt/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [1]:
# Method B: Create dataset from generator.
import datasets # type: ignore
n = 1_000_000
generator = lambda: ({"question": "A number?", "answer": i} for i in range(n))
ds = datasets.Dataset.from_generator(generator)
print(ds)
print(ds[0])
print(ds[-1])

/home/zel/ml-projects/ShortGPT/.venv-shortgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'answer'],
    num_rows: 1000000
})
{'question': 'A number?', 'answer': 0}
{'question': 'A number?', 'answer': 999999}


In [3]:
import datasets  # type: ignore

def chunk_generator(lst, chunk_size=25):
    for i in range(0, len(lst), chunk_size):
        yield {"chunk": lst[i:i + chunk_size]}

# Example usage:
data = list(range(100))  # Sample data

# Create dataset from generator
ds = datasets.Dataset.from_generator(lambda: chunk_generator(data))

# Print the dataset
print(ds)

Generating train split: 4 examples [00:00, 673.41 examples/s]

Dataset({
    features: ['chunk'],
    num_rows: 4
})
